# TfIdf-Samples

In [4]:
# Install
#!pip install -r requirements.txt
#pip install numpy==1.20.0
#pip install pandas
#pip install yellowbrick

In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

## Data

In [13]:
# Importing prepared dataset
import pickle
input = open('my_corpus-21-sql.pql', 'rb')
obj = pickle.load(input)
input.close()

txt = obj['txt']
corpus = obj['corpus']
all_words = obj['all_words']
print(corpus[1])

світовий пандемія принести невизначеність вимушений реагувати надшвидко маючи право помилка бізнес винаходити модель благодійний адаптуватися кризовий умова пропонуючи підхід рішення проєкт травень директорка виступити віртуальний саміт благодійництво тема фандрайзинг специфіка ковіднога долучитися дискусіїй секція благодійник бізнес адаптивність криза запрошувати прийняти саміт участь безкоштовний реєстрація посилання


In [14]:
txt.reset_index(drop=True,inplace=True)

## Splitting

In [15]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
c1, c2 = train_test_split(corpus, test_size=0.5, random_state=10)

In [16]:
# TfidfVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(min_df=0.001, use_idf=True, ngram_range=(1,3))
                            #max_df=0.8, max_features=10000
tfidf_matrix1 = vectorizer.fit_transform(c1)
tfidf_matrix2 = vectorizer.fit_transform(c2)
#terms = vectorizer.get_feature_names()

# K-Means

In [8]:
#!pip install clusteval

In [17]:
# Number of Clusters
k = 9

In [32]:
# K-means clustering
from sklearn.cluster import KMeans

km1 = KMeans(n_clusters = k).fit(tfidf_matrix1)
km2 = KMeans(n_clusters = k).fit(tfidf_matrix2)

# Comparing Clustering Algorithms
from sklearn.metrics.cluster import adjusted_rand_score
(adjusted_rand_score(km1.labels_.tolist(),km2.labels_.tolist()).round(3))*100

-0.1

## LDA

In [20]:
from sklearn.decomposition import LatentDirichletAllocation
lda_model = LatentDirichletAllocation(n_components = k,           # Number of topics
                                      max_iter = 20,               # Max learning iterations
                                      random_state = 100,          # Random state
                                      batch_size = 128,            # n docs in each learning iter
                                      evaluate_every = -1,       # compute perplexity every n iters, default: Don't
                                      n_jobs = -1,               # Use all available CPUs
                                     )
lda_output1 = lda_model.fit_transform(tfidf_matrix1)
lda_output2 = lda_model.fit_transform(tfidf_matrix2)

## LDA-K-means

In [33]:
# K-means clustering
kmLDA1 = KMeans(n_clusters = k).fit(lda_output1)
kmLDA2 = KMeans(n_clusters = k).fit(lda_output2)

# Comparing Clustering Algorithms
(adjusted_rand_score(kmLDA1.labels_.tolist(),kmLDA2.labels_.tolist()).round(3))*100

-0.4

## LDA AgglomerativeClustering

In [34]:
from sklearn.cluster import AgglomerativeClustering
hc1 = AgglomerativeClustering(n_clusters = k, affinity = 'euclidean', linkage = 'ward').fit_predict(lda_output1)
hc2 = AgglomerativeClustering(n_clusters = k, affinity = 'euclidean', linkage = 'ward').fit_predict(lda_output2)

# Comparing Clustering Algorithms
(adjusted_rand_score(hc1,hc2).round(3))*100

-0.5

## HDBSCAN

In [35]:
import hdbscan
hdb1 = hdbscan.HDBSCAN(min_cluster_size=5, min_samples=12).fit(tfidf_matrix1)
hdb2 = hdbscan.HDBSCAN(min_cluster_size=5, min_samples=12).fit(tfidf_matrix2)

# Comparing Clustering Algorithms
(adjusted_rand_score(hdb1.labels_,hdb2.labels_).round(3))*100

0.1

In [30]:
len(set(hdb2.labels_))

9